<a href="https://colab.research.google.com/github/kuansheng-yao/Educational-Data-Mining-Projects/blob/main/%E3%80%8C%E5%AE%8C%E6%95%B4%E6%A7%8B%E6%80%9D%E7%9A%84%E7%89%88%E6%9C%AC_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install opencc
! pip install -U -q google.generativeai
! pip install transformers
! pip install transformers datasets
! git config --global credential.helper store
! pip install diffusers --upgrade
! pip install gtts
! pip install pydub
# ! pip install invisible_watermark transformers accelerate safetensors
# ! pip install --upgrade --quiet pip
# ! pip install --quiet git+https://github.com/huggingface/transformers.git

In [2]:
!pip install 'llama-index==0.9.37' 'google-generativeai==0.3.2' matplotlib qdrant_client

  Using cached google_generativeai-0.3.2-py3-none-any.whl (146 kB)
  Using cached google_ai_generativelanguage-0.4.0-py3-none-any.whl (598 kB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.4
    Uninstalling google-ai-generativelanguage-0.6.4:
      Successfully uninstalled google-ai-generativelanguage-0.6.4
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.6.0
    Uninstalling google-generativeai-0.6.0:
      Successfully uninstalled google-generativeai-0.6.0


In [4]:
! pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
#test image url:https://img.onl/8mQXU4
#test poem= 野幕蔽瓊筵，羌戎賀勞旋。
from opencc import OpenCC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import google.generativeai as genai
import google.ai.generativelanguage as glm
from google.colab import userdata
from IPython.display import Markdown #用來印出data的
# from transformers import BarkModel
import torch
from transformers import AutoProcessor
from IPython.display import Audio
# def text_to_speech(text):
#   model = BarkModel.from_pretrained("suno/bark-small")
#   device = "cuda:0" if torch.cuda.is_available() else "cpu"
#   model = model.to(device)
#   processor = AutoProcessor.from_pretrained("suno/bark")
#   # prepare the inputs
#   inputs = processor(text)
#   # generate speech
#   speech_output = model.generate(**inputs.to(device))
#   sampling_rate = model.generation_config.sample_rate
#   display(Audio(speech_output[0].cpu().numpy(), rate=sampling_rate))

#text_to_speech
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import re
import os

def insert_pauses(text):
    pauses = {
        ' ': 300,
        ',': 200,
        '.': 500,
        '?': 600,
        '!': 600,
        '，': 200,
        '。': 500,
        '？': 600,
        '！': 600
    }

    for punct, pause in pauses.items():
        text = text.replace(punct, f'<PAUSE={pause}>')

    return text

def text_to_speech(text, lang='zh', slow=False):
    processed_text = insert_pauses(text.strip())  # 去除開頭和結尾的空白

    combined_audio = AudioSegment.empty()
    segments = re.split(r'(<PAUSE=\d+>)', processed_text)

    for segment in segments:
        if segment.startswith('<PAUSE='):
            pause_duration = int(re.search(r'\d+', segment).group())
            combined_audio += AudioSegment.silent(duration=pause_duration)
        else:
            if segment.strip():  # 確保文本段落不是空白
                tts = gTTS(text=segment, lang=lang, slow=slow)
                tts.save('temp_segment.mp3')
                audio_segment = AudioSegment.from_mp3('temp_segment.mp3')
                combined_audio += audio_segment

    output_file = "output_with_pauses.mp3"
    combined_audio.export(output_file, format='mp3')
    os.remove('temp_segment.mp3')
    display(combined_audio)

def text_to_image(text):
  from diffusers import DiffusionPipeline
  import torch
  import IPython.display
  pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
  pipe.enable_model_cpu_offload()
  #prompt 裡輸入遇轉換成圖片的文字（需要是英文）
  images = pipe(prompt=text).images[0]
  display(images)


# model configuring
API_KEY=userdata.get('API_KEY')
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-pro')

# scraping peom data
def scrape_poem_data():
  import requests
  from bs4 import BeautifulSoup
  num=["001","034","041","070","083","163","216","217","246","254","305"]
  data_store=""
  converter = OpenCC('s2twp')
  for i in range(1,12):
      if i<10:
          url = f"https://www.millionbook.net/gd/h/hengtangtuishi/tssb/00{i}.htm#{num[i-1]}"
      else:
          url = f"https://www.millionbook.net/gd/h/hengtangtuishi/tssb/0{i}.htm#{num[i-1]}"
      response = requests.get(url)
      response.encoding = 'big5'
      content = response.text
      result = BeautifulSoup(content, "html.parser")
      data=result.find_all("span",{"class":"swy1"})
      for i in data:
        t=converter.convert(i.text)
        data_store+=t
  lst=data_store.split("=============================")
  documents = lst[1:]
  documents = [s.replace('\r', '').replace('\n', ' ') for s in documents]
  return documents

#prompt templete
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""你現在是一個中文學習者的小幫手，你需要根據下方提供的段落幫忙回答有關唐詩的問題。請盡量以口語、好懂的方式表達，這樣可以幫助學習者更容易理解。
  如果提供的段落無關問題你可以忽略。
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

# CV
def read_image(url):
  from llama_index.multi_modal_llms.gemini import GeminiMultiModal

  from llama_index.multi_modal_llms.generic_utils import (
      load_image_urls,)
  import os
  from google.colab import userdata
  #Setup GEMINI API KEY in Environment variable of Runtime
  # add your GOOGLE API key here
  GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
  os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
  image_urls = [url]
  image_documents = load_image_urls(image_urls)
  gemini_pro = GeminiMultiModal(model_name="models/gemini-pro-vision")
  complete_response = gemini_pro.complete(

      prompt="請讀出圖片中的文字，這是一首詩。",
      image_documents=image_documents,
  )
  complete_response=str(complete_response.text).replace("\n","，").replace(" ","")
  return complete_response

# main
searching_choice=input("input \"1\" if you want to insert a poem with text or input \"2\" if you wanna upload a image")
if searching_choice == str(1):
  search_poem=input("請輸入想要查找的詩。Please input a poem you want to search")
  language=input("the language you want to translate to:")
  query = f"請給我這首詩的詩名、作者、解析以及這首詩想表達的意義，除了以繁體中文表示外，請同時將所有的內容翻譯成{language}。:{search_poem}"

  #find the most relevant passage in the documents
  # Convert the given passage and list of passages into TF-IDF vectors
  documents= scrape_poem_data()
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(documents + [search_poem])

  # Calculate cosine similarity between the given passage and each passage in the list
  similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

  # Find the index of the most relevant passage
  most_relevant_index = similarities.argmax()

  # Print the most relevant passage
  print("Most relevant passage:")
  print(documents[most_relevant_index])

  #pass a query to prompt
  prompt = make_prompt(query, documents[most_relevant_index])

  #use the generate_content method to generate a reponse from the model base on the prompt
  model = genai.GenerativeModel('gemini-pro')
  answer = model.generate_content(prompt)
  print(answer.text)
  import re
  text = documents[most_relevant_index]
  # Define the regular expression pattern to match words before "【註解】"
  pattern = r'(.+?)(?=【註解】)'
  # Search for the pattern in the text
  match = re.search(pattern, text)
  if match:
      extracted_words = match.group(1)
      print("Extracted words:", extracted_words)

  text_to_speech(extracted_words)
  #text_to_image
  pattern = r'【韻譯】：(.*?)【評析】'
  match = re.search(pattern,text)
  if match:
    extracted_text = match.group(1)
    model = genai.GenerativeModel('gemini-pro')
    prompt = f"please translate the following sentences into English with simple and conscise way:｛extracted_text｝"
    answer = model.generate_content(prompt)
    text_to_image(answer.text)


else:
  search_poem=input("上傳包含詩詞的圖片的URL。Please upload an URL of an image including a poem")
  poem=read_image(search_poem)
  language=input("the language you want to translate to:")
  query = f"請給我這首詩的詩名、作者、解析以及這首詩想表達的意義，除了以繁體中文表示外，請同時將所有的內容翻譯成{language}。:{poem}"

  #find the most relevant passage in the documents
  # Convert the given passage and list of passages into TF-IDF vectors
  documents= scrape_poem_data()
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(documents + [poem])

  # Calculate cosine similarity between the given passage and each passage in the list
  similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

  # Find the index of the most relevant passage
  most_relevant_index = similarities.argmax()

  # Print the most relevant passage
  print("Most relevant passage:")
  print(documents[most_relevant_index])

  #pass a query to prompt
  prompt = make_prompt(query, documents[most_relevant_index])

  #use the generate_content method to generate a reponse from the model base on the prompt
  model = genai.GenerativeModel('gemini-pro')
  answer = model.generate_content(prompt)
  print(answer.text)

  import re
  text = documents[most_relevant_index]
  # Define the regular expression pattern to match words before "【註解】"
  pattern = r'(.+?)(?=【註解】)'
  # Search for the pattern in the text
  match = re.search(pattern, text)
  if match:
      extracted_words = match.group(1)
      print("Extracted words:", extracted_words)
  text_to_speech(extracted_words)
  #text_to_image
  pattern = r'【韻譯】：(.*?)【評析】'
  match = re.search(pattern,text)
  if match:
    extracted_text = match.group(1)
    model = genai.GenerativeModel('gemini-pro')
    prompt = f"please translate the following sentences into English with simple and conscise way:｛extracted_text｝"
    answer = model.generate_content(prompt)
    text_to_image(answer.text)





input "1" if you want to insert a poem with text or input "2" if you wanna upload a image1
請輸入想要查找的詩。Please input a poem you want to search送孟浩然之廣陵
the language you want to translate to:中文
Most relevant passage:
 《送孟浩然之廣陵》 作者：李白 故人西辭黃鶴樓，煙花三月下揚州。 孤帆遠影碧空盡，惟見長江天際流。   【註解】： １、黃鶴樓：建在湖北武昌西邊的黃鶴磯上，下面就是長江。 ２、煙花：指暮春濃艷的景色。 ３、碧空盡：指船消失在水與藍天相接的地方。  【韻譯】： 老朋友孟浩然，辭別西楚的黃鶴樓； 陽春三月煙花如海，他去遊歷揚州。 一葉孤舟，遠遠地消失在碧空盡頭； 只見浩浩蕩蕩的長江，向天際奔流！  【評析】： �U�U這是送別詩，寓離情於寫景。首句點出送別的地點：一代名勝黃鶴樓；二句寫送 別的時間與去向：“煙花三月”的春色和東南形勝的“揚州”；三、四句，寫送別的 場景：目送孤帆遠去；只留一江春水。詩以絢麗斑駁的煙花春色和浩瀚無邊的長江為 背景，極盡渲染之能事，繪出了一幅意境開闊，情絲不絕，色彩明快，風流倜儻的詩 人送別畫。“煙花三月下揚州”，蘅塘退士評曰：“千古麗句”。在理。    
**詩名：** 送孟浩然之廣陵

**作者：** 李白

**解析：**

這是一首送別的詩，作者李白送他的朋友孟浩然去廣陵。

第一句「故人西辭黃鶴樓」說孟浩然從西邊的黃鶴樓出發。
第二句「煙花三月下揚州」描寫當時春光明媚，孟浩然要去的揚州景色優美。
第三句「孤帆遠影碧空盡」寫孟浩然的船漸漸遠去，消失在天邊。
第四句「惟見長江天際流」寫送別的人仍然看到長江朝著天際流去。

**意義：**

這首詩表達了作者對朋友的依依不捨之情。作者用美麗的景色來襯托離別的場景，更讓讀者感受到送別的傷感。
Extracted words:  《送孟浩然之廣陵》 作者：李白 故人西辭黃鶴樓，煙花三月下揚州。 孤帆遠影碧空盡，惟見長江天際流。   


/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-66600e4f-031726c52c716c71412715af;adeb373c-acd0-42e1-ae2d-fb2622a25c4e)

Repository Not Found for url: https://huggingface.co/api/models/stabilityai/stable-diffusion-xl-base-1.0.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
User Access Token "mistral" is expired.
Will try to load from local cache.


OSError: Cannot load model stabilityai/stable-diffusion-xl-base-1.0: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.